**1 Install and Import Libs**

In [10]:
!pip install llama-index-llms-anthropic -q
!pip install llama-index-embeddings-openai -q
!pip install llama-index -q
!pip install anthropic -q
!pip install openai -q

In [11]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab_Notebooks/Agent

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab_Notebooks/Agent


In [12]:
from llama_index.llms.anthropic import Anthropic
from llama_index.core.tools import FunctionTool
import nest_asyncio
nest_asyncio.apply()
from google.colab import userdata
CLAUDE_API_KEY = userdata.get('ANTHROPIC_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
LLAMA_CLOUD_API_KEY = userdata.get('LLAMAPARSE_API_KEY')
import openai
openai.api_key = OPENAI_API_KEY

**Load documents**

In [13]:
import os
path_pdf = os.path.join(os.getcwd(),'jpmorgan_annualreport-2022-splitted.pdf')
output_directory_path = os.path.join(os.getcwd(),'test_claude')

**2 RAG**

**2.1 LlamaParse: Parse Document using markdown parser**

In [14]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_parse import LlamaParse
pdf_file = os.path.join(os.getcwd(),'jpmorgan_annualreport-2022-splitted.pdf')
# set up parser
parser = LlamaParse(api_key=LLAMA_CLOUD_API_KEY, result_type="markdown")
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_files=[pdf_file], file_extractor=file_extractor).load_data()
# Index the documents in a vectorStore
index_jpmorgan = VectorStoreIndex.from_documents(documents)

Started parsing the file under job_id 7c77cbb8-1542-4572-aa75-51fc463c8db0
.

In [15]:
documents

[Document(id_='94663e86-c014-4942-bbde-fcca6960cfd0', embedding=None, metadata={'file_path': '/content/drive/MyDrive/Colab_Notebooks/Agent/jpmorgan_annualreport-2022-splitted.pdf', 'file_name': 'jpmorgan_annualreport-2022-splitted.pdf', 'file_type': 'application/pdf', 'file_size': 103636, 'creation_date': '2024-11-17', 'last_modified_date': '2024-11-17'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='# Earnings, Diluted Earnings per Share and Return on Tangible Common Equity\n\n# 2004–2022\n\n($ in billions, except per share and ratio data)\n\n| |Reported|Reported|Reported|Excluding reserve release/build1|Excluding reserve release/build1|Excluding reserve release/build1|\n|---|---|---|\n| |2020|2021|2022|2020|2021|2022|\n|Net income ($B)|$29.1|$48.3|$3

**2.2 Query the Document using GPT-4-Turbo**

In [16]:
from llama_index.llms.openai import OpenAI
llm_openai = OpenAI(model="gpt-4-turbo", temperature=0.1)
query_engine_jpmorgan = index_jpmorgan.as_query_engine(llm=llm_openai)
# query the engine
query = "What was the Net Income in 2020, the Reported one?"
response_jpmorgan = query_engine_jpmorgan.query(query)
print("*"*50)

**************************************************


In [17]:
print(f"The answer to the question '''{query}''' is :\n")
response_jpmorgan.response

The answer to the question '''What was the Net Income in 2020, the Reported one?''' is :



'The reported net income for 2020 was $29.1 billion.'

**3 Build an Anthropic Agent over RAG**

**3.1 RAG tool**

FunctionCallingAgentWorker using the query engine tool and initialize the Anthropic
Agent.

In [18]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata
#Create the Query Engine Tool over the RAG pipeline
query_engine_tool = QueryEngineTool(
query_engine=query_engine_jpmorgan,
metadata=ToolMetadata(
name="jpmorgan_annualreport-2022",
description=(
"Provides Annual Report information about JPMorgan for 2022. "
"Use a detailed plain text question as input to the tool."),),)

In [23]:
from llama_index.core.agent import FunctionCallingAgentWorker
llm_anthropic = Anthropic(model="claude-3-opus-20240229",api_key=CLAUDE_API_KEY)
agent_worker = FunctionCallingAgentWorker.from_tools(
[query_engine_tool], llm=llm_anthropic, verbose=True
)
agent = agent_worker.as_agent()

**3.2 Chat with the Agent**

In [ ]:
query = "What was the Net Income in 2020, the Reported one?"
response = agent.chat(query)
print(str(response))

In [ ]:
query = "What was the evolution of the Net Income from 2020 to 2021, the Reported one?"
response = agent.chat(query)
print(str(response))

In [ ]:
query = "What was the evolution in term of percentage of the Net Income from 2020 to 2021, the Reported one?"
response = agent.chat(query)
print(str(response))

**3.3 Add another Tool to the Agent:**


Add another Tool to the Agent:
• Calculate the evolution rom year X to year Y
• Calculate Compound Annual Growth Rate
• Calculate Price Per Earning Ratio

In [ ]:
def evolution(a: float, b: float) -> float:
  """Evolution in percentage from value b to value a"""
  return f"{round(100*(a/b-1),0)}%"
evolution_tool = FunctionTool.from_defaults(fn=evolution)

In [ ]:
def cagr(a: float, b: float, n: int) -> float:
  """Compound annual growth rate CAGR from value b to value a over n period"""
  return f"{round(100*((a/b)^(1/n)-1),0)}%"
cagr_tool = FunctionTool.from_defaults(fn=cagr)

In [ ]:
def price_earning_ratio(price: float, eps: float) -> float:
  """Price per earning ratio is a ratio of the share price devided by the earning per share(EPS) value"""
  return price/eps
pe_tool = FunctionTool.from_defaults(fn=price_earning_ratio)

**3.4  Update the Agent with this second tool**

In [ ]:
agent_worker2 = FunctionCallingAgentWorker.from_tools([query_engine_tool, evolution_tool], llm=llm_anthropic, verbose=True)
agent2 = agent_worker2.as_agent()
query = "What was the evolution of the Net Income from 2020 to 2021, the Reported one?"
response = agent2.chat(query)
print(str(response))

In [ ]:
query = "What was the evolution of the Net Income from 2021 to 2022, the Reported one?"
response = agent2.chat(query)
print(str(response))

In [ ]:
query = "Compare the evolution of the Net Income between 2020 to 2021 and 2021 to 2022, the Reported one."
response = agent2.chat(query)
print(str(response))

In [ ]:
agent_worker3 = FunctionCallingAgentWorker.from_tools(
[query_engine_tool, evolution_tool, cagr_tool], llm=llm_anthropic,verbose=True)
agent3 = agent_worker3.as_agent()
query = "What was the compound annual growth of the Net Income from 2020 to 2022, the Reported one?"
response = agent3.chat(query)
print(str(response))

In [ ]:
agent_worker3 = FunctionCallingAgentWorker.from_tools(
[query_engine_tool, evolution_tool, cagr_tool, pe_tool], llm=llm_anthropic,verbose=True)
agent3 = agent_worker3.as_agent()
query = "knowning that the price of JPMorang in 2022 was $134.10 and given the financial report,what was the price/earning ratio?"
response = agent3.chat(query)
print(str(response))